7.2 
Compute Precision, Recall and F1 values for the following two parses


In [3]:
#labeled
print("Labeled:")
# 14 pos-tags, 5 segments
sys = 19

#14 pos, 4 segs
gold = 18

# 14 pos, 4 segs
tp= 18

# precision (tp / tp +fp)
precision =tp / sys
print(f"Precision: {precision}")

# recall tp / tp + fn
recall = tp / gold
print(f"Recall: {recall}")

f1 = (2* precision * recall)/(precision + recall)
print(f"F1: {f1}")

Labeled:
Precision: 0.9473684210526315
Recall: 1.0
F1: 0.972972972972973


In [4]:
print("unlabeled")
# 8 left S, 1 konj, 7 right s
sys = 16
# 8 + 7
gold = 15

tp = 8
# precision (tp / tp +fp)
precision =tp / sys
print(f"Precision: {precision}")

# recall tp / tp + fn
recall = tp / gold
print(f"Recall: {recall}")

f1 = (2* precision * recall)/(precision + recall)
print(f"F1: {f1}")

unlabeled
Precision: 0.5
Recall: 0.5333333333333333
F1: 0.5161290322580646


Is it possible to get 0% unlabeled precision?  
Yes, e.g. if all parts are directly linked to the main s while in the gold set every word is part of a substructure (NP) of s

72b Train PCFG  


In [37]:
class MLEEntry:
    def __init__(self, tag):
        self.count = 0
        self.tag = tag
        self.connections = 0
        self.tag_con = {}
    
    def add(self, tag, count=1):
        try:
            self.tag_con[tag] += count
        except KeyError:
            self.tag_con[tag] = count
        self.connections += count
    
    def info(self):
        print(f'Tag: {self.tag}')
        print(f"Count: {self.count}")
        print(f"Connections: {self.connections}")
        print(self.tag_con)
            
s = MLEEntry("s")
np = MLEEntry("np")
pp = MLEEntry("pp")
pn = MLEEntry("pn")

In [38]:
#counts außerdedem gehören...
s.count += 1
s.add("adv", 1)
s.add("vvfin", 1)
s.add("fper", 1)
s.add("np",1)

np.count += 6
np.add("np", 5)
np.add("pn",1)
np.add("art",1)
np.add("pp",1)
np.add("nn", 6)
np.add("kon",2)
np.add("adj", 1)
np.add("kokom", 1)

pn.count += 1
pn.add("ne",1)

pp.count +=1
pp.add("appr", 1)
pp.add("adja",1)
pp.add("nn",1 )

In [39]:
# counts so erklärt...
s.count += 1
s.add("adv", 1)
s.add("vvfin", 1)
s.add("np", 1)

np.count += 1
np.add("pn", 1)
np.add("pp",1)

pn.count += 2
pn.add("ne", 4)

pp.count += 2
pp.add("appr", 2)
pp.add("art", 1)
pp.add("nn", 1)
pp.add("pn", 1)
pp.add("pp", 1)
pp.add("ne", 1)


In [40]:
# counts er wäre vielleich..

s.count += 3
s.add("s", 2)
s.add("kon", 1)
s.add("pper", 1)
s.add("vafin", 2)
s.add("adv", 1)
s.add("np", 2)
s.add("pds",1)
s.add("ptkneg", 1)

np.count += 2
np.add("art", 1)
np.add("adja", 1)
np.add("nn", 1)
np.add("pposat",1)
np.add("nn", 1)

In [41]:
# counts die kritik...

s.count += 1
s.add("np", 1)
s.add("vvfin", 1)
s.add("prf", 1)
s.add("pp", 2)

np.count += 2
np.add("art",2)
np.add("nn", 2)
np.add("np", 1)
np.add("pp", 1)

pp.count += 4
pp.add("appr", 2)
pp.add("ne",1)
pp.add("pds",1)
pp.add("adja",1)
pp.add("nn", 2)
pp.add("appart",1)

In [42]:
s.info()

Tag: s
Count: 6
Connections: 23
{'adv': 3, 'vvfin': 3, 'fper': 1, 'np': 5, 's': 2, 'kon': 1, 'pper': 1, 'vafin': 2, 'pds': 1, 'ptkneg': 1, 'prf': 1, 'pp': 2}


In [43]:
np.info()

Tag: np
Count: 11
Connections: 31
{'np': 6, 'pn': 2, 'art': 4, 'pp': 3, 'nn': 10, 'kon': 2, 'adj': 1, 'kokom': 1, 'adja': 1, 'pposat': 1}


In [44]:
pp.info()

Tag: pp
Count: 7
Connections: 18
{'appr': 5, 'adja': 2, 'nn': 4, 'art': 1, 'pn': 1, 'pp': 1, 'ne': 2, 'pds': 1, 'appart': 1}


In [45]:
pn.info()

Tag: pn
Count: 3
Connections: 5
{'ne': 5}


In [48]:
mles = [s, np, pp, pn]
def get_pcfg(mles):
    for mle in mles:
        for key in mle.tag_con.keys():
            print(f"{mle.tag} ---> {key}: { mle.tag_con[key] / mle.connections}")

In [49]:
get_pcfg(mles)

s ---> adv: 0.13043478260869565
s ---> vvfin: 0.13043478260869565
s ---> fper: 0.043478260869565216
s ---> np: 0.21739130434782608
s ---> s: 0.08695652173913043
s ---> kon: 0.043478260869565216
s ---> pper: 0.043478260869565216
s ---> vafin: 0.08695652173913043
s ---> pds: 0.043478260869565216
s ---> ptkneg: 0.043478260869565216
s ---> prf: 0.043478260869565216
s ---> pp: 0.08695652173913043
np ---> np: 0.1935483870967742
np ---> pn: 0.06451612903225806
np ---> art: 0.12903225806451613
np ---> pp: 0.0967741935483871
np ---> nn: 0.3225806451612903
np ---> kon: 0.06451612903225806
np ---> adj: 0.03225806451612903
np ---> kokom: 0.03225806451612903
np ---> adja: 0.03225806451612903
np ---> pposat: 0.03225806451612903
pp ---> appr: 0.2777777777777778
pp ---> adja: 0.1111111111111111
pp ---> nn: 0.2222222222222222
pp ---> art: 0.05555555555555555
pp ---> pn: 0.05555555555555555
pp ---> pp: 0.05555555555555555
pp ---> ne: 0.1111111111111111
pp ---> pds: 0.05555555555555555
pp ---> appart: 0.